Let's start with good old linear regression model. 

$ Y = X\beta + \epsilon $

All of the members of the equation are matrices. Their dimensions are useful to know for the following calculations, so let's write them out:  

$Y (nx1)$  
$X (nxk)$  
$\beta (kx1)$  
$\epsilon (nx1)$  

We have an interest in the realtionship between Xs and Y, thus we need to find $\beta$
One of the classical ways is to find it my minimizing the square error ($\epsilon$) of the equation. The square error can be expressed like this  
$\epsilon^T\epsilon = (Y-X\beta)^T (Y-X\beta)  \equiv L$

Notice that $\epsilon^T\epsilon$ is scalar value (1xn) (nx1) which can be minimized and it also implies that we are dealing with scalars on the right hand side. (I told you than knowing the dimensions would come in handy)

$L = YY^T - Y^TX\beta - X^T\beta^TY + \beta^TX^TX\beta  $ (0)

The second and third term are equvalent in value so we just can multiply any of them by 2.

$$L = YY^T - 2X^T\beta^TY + \beta^TX^TX\beta  $$

Take the derivative and equate it to zero

$\frac{\partial L}{\partial \beta}  = - 2X^TY - 2X^TX\beta = 0  $ (1)  
$ X^TX\beta = X^TY  $  
$ \beta = (X^TX)^{(-1)}X^TY $  (2)

A close form solution in thus found for $\beta$ we can see it is nothing more than a combination of Xs and Ys.

Let's do an example of a regression in the form of $y = 5.01 + 2.1*x_1 - 0.5*x_2 + \epsilon$

In [46]:
# An example
import numpy as np

np.random.seed(42)
x1=np.array(range(0, 50)) + np.random.normal(size=(50))
x2=np.random.normal(size=(50))
x=np.stack((x1, x2)).T
X = np.c_[np.ones((len(x), 1)), x]

y = 5.01 + 2.1*x1 - 0.5*x2 + np.random.normal(loc=0.0, scale=0.2, size=50)

In [47]:
#Equation (2)
np.linalg.inv(X.T @ X) @ X.T @ y
#The solution is pretty close the values stated in the previous cell. The method works :)

array([ 5.02609659,  2.09905363, -0.55519427])

Now let's see how the parameter estimation look like if esimate it through gradient descent.

We have the equation (1) we can use  
$\frac{d L}{d \beta} = \nabla f = - 2X^TY - 2X^TX\beta  $
Let's just rewrite $X\beta = \hat{Y}$

$\nabla f   = - 2X^T(Y-\hat{Y}) = 2X^T(\hat{Y}-Y)   $ (3)

And we thus have the gradient. But now the question remains on how to update the $\beta$. Usully the equation is given without the derivation as  
$\beta_{t+1} = \beta_{t} - lr*\frac{\partial L}{\partial \beta} $ (4) (t here denotes learing steps, meaning t+1 is the updated $\beta$)

However, in order to completely understand what is going on we can derive equation (4) 

We'll need Taylor's series expansion. Let's first remember how it look like in the general case and later use for gradient descent. Taylor series lets us to approximate a function locally by a polynomial order of our choosing. Let's say we have funciton $f(x)$ and expand it in point $x_{0}$

$f(x) = \frac{f(x{_0})}{0!}(x-x_{0})^0 + \frac{f'(x{_0})}{1!}(x-x_{0})^1 + \frac{f''(x{_0})}{2!}(x-x_{0})^2 + \xi = \frac{f^n(x{_0})}{n!}(x-x_{0})^n$

Now let's do it for the function of $f(x+\Delta x)$ in the point $x$

$f(x+\Delta x) \approx f(x) + f'(x{_0})\Delta x$

now let's just replace our variables $\beta_{t+1} \equiv f( x+\Delta x) $ and $\beta_{t} \equiv x $

$f(\beta_{t+1}) \approx f(x) + f'(\beta_{t})\Delta x$



So to summarize the process of gradient descent looks like this:

0. Guess/initialize values of $\beta$ to some (random) numbers
1. Calculate the $\hat{Y}$ with the current $\beta$ (aka $\beta_t$)
2. Calculate the gradient with requires $\hat{Y}$ we calculated in the first step which in turn requires the $\beta$ 
3. Update the $\beta_{t}$ using the equation (4), thus calculating  $\beta_{t+1}$
4. Repeat the steps 1-3 updating the $\beta$ values until they stop improving (some tolerance level is reached)

To make even clearer here is a schema on how the whole process works 
$$\beta_t \xrightarrow{\text{calculate}} {\hat{Y_t} \xrightarrow{\text{calculate}} \nabla f \xrightarrow{\text{update}} \beta_{t+1} }$$



In [48]:
def gradient_descent(X, Y, add_intercept = True, learningrate = 0.001, iterations = 100000):
    y_real = np.reshape(Y, (len(Y), 1))   
    cost_list = []
    m=X.shape[0]

    if add_intercept:
        X = np.c_[np.ones((len(X), 1)), X]

    beta = np.random.randn(X.shape[1], 1)
    
    for i in range(iterations):
        #Equation (3)
        gradient = 2/m * X.T.dot(X.dot(beta) - y_real)
        #Equation (4)
        beta = beta - learningrate * gradient
        y_hat = X.dot(beta)
        #Equation (0)
        cost_value = 1/(2*m)*((y_hat - y)**2) 
        total = 0
        for i in range(len(y)):
            total += cost_value[i][0] 
        cost_list.append(total)
    return beta

In [49]:
gradient_descent(x, y, add_intercept = True, learningrate = 0.001, iterations = 100000)

array([[ 5.02609659],
       [ 2.09905363],
       [-0.55519427]])

The result is pretty close one again, so the algorithm works well at least for this case

${\large Appendix}$

More derivation without using matrix form, it may better explain the `gradient_descent` function above

$$L(\beta) = \frac{1}{2m} \sum_{i=1}^{m} (h(x^{(i)}) - y^{(i)})^2$$

where:

$m$ is the number of training examples
$x^{(i)}$ and $y^{(i)}$ are the $i^{th}$ input-output pair
$h(x) = \beta_0 + \beta_1 x_1 + ... + \beta_n x_n$ is the hypothesis function represented by the linear model
$\beta$ is the vector of parameters $(\beta_0, \beta_1, ..., \beta_n)$
Gradient of the cost function:

$$\nabla L(\beta) = \frac{1}{m} \sum_{i=1}^{m} (h(x^{(i)}) - y^{(i)}) x^{(i)}$$

Parameter update rule:

$$\beta_{t+1} = \beta_t - \alpha \frac{\partial L(\beta)}{\partial \beta_t}$$